In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
from importlib import reload
from datetime import datetime

sys.path.insert(0, '../src')
import deepleeo.dataset.data_augment as dtaug
import deepleeo.dataset.utils as dsutils 
import deepleeo.utils.geofunctions as gf
import deepleeo.networks.model_builder as mb

reload(dtaug)
reload(dsutils)
reload(mb)
reload(gf)

/home/raian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'deepleeo.utils.geofunctions' from '../src/deepleeo/utils/geofunctions.py'>

# Load input Dataset

In [2]:
# DATA_DIR = os.path.join(os.path.abspath(os.path.dirname("__file__")), '../', 'data_real', 'generated')
network = "fcn8s"
DATA_DIR = "/home/raian/doutorado/Dados/generated"
DATASET_FILE = os.path.join(DATA_DIR, 'dataset_286x286_timesstack-2015-2016.npz')#'dataset_1.npz')

# model_dir = os.path.join(DATA_DIR, 'tf_logs', "test_%s_%s" % (network, datetime.now().strftime('%d_%m_%Y-%H_%M_%S')))
model_dir = "/home/raian/doutorado/Dados/generated/tf_logs/test_debug"
# model_dir = os.path.join(DATA_DIR, 'tf_logs', 'test_fcn8s_11_12_2018-17_26_32')

In [3]:
!rm -rf {model_dir}/*
!ls -al {model_dir}

total 8
drwxr-xr-x  2 raian digits 4096 Mar 18 20:05 .
drwxr-xr-x 15 raian digits 4096 Feb 22 22:26 ..


In [4]:
dataset = np.load(DATASET_FILE)

print("Data Loaded:")
print("  -> Images: ", len(dataset["images"]))
print("  -> Labels: ", len(dataset["labels"]))
print("  -> Classes: ", len(dataset["classes"]))

print("Images shape: ", dataset["images"][0].shape, " - DType: ", dataset["images"][0].dtype)
print("Labels shape: ", dataset["labels"][0].shape, " - DType: ", dataset["labels"][0].dtype)
# print("UNIQUE LABELS: ", np.unique(dataset["labels"]))

Data Loaded:
  -> Images:  5000
  -> Labels:  5000
  -> Classes:  2
Images shape:  (286, 286, 10)  - DType:  float32
Labels shape:  (286, 286, 1)  - DType:  int32


# Perform Data Augmentation

In [5]:
# reload(dtaug)
# new_dataset = {}
# new_dataset["images"] = dataset["images"]
# new_dataset["labels"] = dataset["labels"]

# steps = 1000
# num_chips = len(dataset["images"])
# for i in range(0, num_chips, steps):
#     endstep = i + steps
#     if endstep > num_chips:
#         endstep = num_chips - 1
    
#     angles = [90, 180, 270]
#     rotated_imgs = dtaug.rotate_images(dataset["images"][i:endstep], angles)
#     flipped_imgs = dtaug.flip_images(dataset["images"][i:endstep])

#     new_dataset["images"] = np.concatenate((new_dataset["images"], rotated_imgs))
#     new_dataset["images"] = np.concatenate((new_dataset["images"], flipped_imgs))

#     rotated_lbls = dtaug.rotate_images(dataset["labels"][i:endstep], angles)
#     flipped_lbls = dtaug.flip_images(dataset["labels"][i:endstep])

#     new_dataset["labels"] = np.concatenate((new_dataset["labels"], rotated_lbls))
#     new_dataset["labels"] = np.concatenate((new_dataset["labels"], flipped_lbls)).astype(dtype=np.int32)

# new_dataset["classes"] = dataset["classes"]

# print("Data Augmentation Applied:")
# print("  -> Images: ", new_dataset["images"].shape)
# print("  -> Labels: ", new_dataset["labels"].shape)

# Split dataset between train, test and validation data

In [6]:
train_images, test_images, valid_images, train_labels, test_labels, valid_labels = dsutils.split_dataset(dataset)

print("Splitted dataset:")
print("  -> Train images: ", train_images.shape)
print("  -> Test images: ", test_images.shape)
print("  -> Validation images: ", valid_images.shape)
print("  -> Train Labels: ", train_labels.shape)
print("  -> Test Labels: ", test_labels.shape)
print("  -> Validation Labels: ", valid_labels.shape)

Splitted dataset:
  -> Train images:  (3500, 286, 286, 10)
  -> Test images:  (1500, 286, 286, 10)
  -> Validation images:  (0, 286, 286, 10)
  -> Train Labels:  (3500, 286, 286, 1)
  -> Test Labels:  (1500, 286, 286, 1)
  -> Validation Labels:  (0, 286, 286, 1)


# Train the Network

In [7]:
params = {
    "epochs": 20,
    "batch_size": 100,
    "learning_rate": 0.0001,
    "l2_reg_rate": 0.5,
    "var_scale_factor": 2.0,
    "chips_tensorboard": 2,
    "dropout_rate": 0.5,
    "class_names": dataset["classes"],
    "bands_plot": [1,2,3]
}

In [8]:
model = mb.ModelBuilder(network)
model.train(train_images, test_images, train_labels, test_labels, params, model_dir)

Instructions for updating:
Please use `tf.contrib.distribute.MirroredStrategy` instead.
INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_is_chief': True, '_train_distribute': None, '_evaluation_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_summary_steps': 100, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1c2c82dfd0>, '_num_worker_replicas': 1, '_eval_distribute': None, '_model_dir': '/home/raian/doutorado/Dados/generated/tf_logs/test_debug', '_master': '', '_keep_checkpoint_max': 5, '_experimental_distribute': None,

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-105
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 105 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:loss = 0.6385089
INFO:tensorflow:loss = 0.6385089, step = 105
INFO:tensorflow:loss = 0.6308788 (12.648 sec)
INFO:tensorflow:loss = 0.6256417 (9.873 sec)
INFO:tensorflow:loss = 0.61745644 (8.766 sec)
INFO:tensorflow:Saving checkpoints for 140 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:Loss for final step: 0.6158496.
---------------
Evaluating...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-18-23:12:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/

INFO:tensorflow:loss = 0.49636075
INFO:tensorflow:loss = 0.4973831 (6.239 sec)
INFO:tensorflow:Finished evaluation at 2019-03-18-23:18:06
INFO:tensorflow:Saving dict for global step 280: eval_metrics/accuracy = 0.7511195, eval_metrics/f1-score = 0.103486545, global_step = 280, loss = 0.49544734
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 280: /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-280
Epoch  9
---------------
Training...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-280
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 280 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:loss = 0.49742603
INFO:tensorflow:loss = 

INFO:tensorflow:loss = 0.35629553 (9.723 sec)
INFO:tensorflow:loss = 0.35029316 (8.437 sec)
INFO:tensorflow:Saving checkpoints for 455 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:Loss for final step: 0.34391922.
---------------
Evaluating...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-18-23:24:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-455
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:loss = 0.3476264
INFO:tensorflow:loss = 0.34882578 (6.741 sec)
INFO:tensorflow:Finished evaluation at 2019-03-18-23:24:46
INFO:tensorflow:Saving dict for global step 455: eval_metrics/accuracy = 0.84409904, eval_metrics/f1-score = 0.12826973, global_step = 455, loss = 0.34520283
INFO:tensorflow:Saving 'checkpoint_path' summary f

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-595
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 595 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:loss = 0.25213328
INFO:tensorflow:loss = 0.25213328, step = 595
INFO:tensorflow:loss = 0.22790833 (12.478 sec)
INFO:tensorflow:loss = 0.23448819 (9.680 sec)
INFO:tensorflow:loss = 0.23140422 (8.537 sec)
INFO:tensorflow:Saving checkpoints for 630 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:Loss for final step: 0.22427893.
---------------
Evaluating...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-18-23:31:12
INFO:tensorflow:Gra

In [9]:
#fcn.fcn_evaluate(valid_images, valid_labels,)